In [1]:
from torch import nn

from multi_modal_edge_ai.adl_inference.data_access.parser import parse_file
from multi_modal_edge_ai.adl_inference.ml_models.cnn_model import CNNModel
from multi_modal_edge_ai.adl_inference.preprocessing.encoder import Encoder
from multi_modal_edge_ai.adl_inference.validating.validate import split_and_validate


(sdf, adf) = parse_file(r'../public_dataset/OrdonezA_Sensors.csv', r'../public_dataset/OrdonezA_ADLs.csv')

adls = adf['Activity'].unique().tolist()
adl_encoder = Encoder(adls)

# If you want to replace the existing 'Activity' column with encoded values
adf['Activity'] = adf['Activity'].apply(adl_encoder.encode_label)

# Assuming df is your DataFrame

sensors = sdf['Sensor'].unique().tolist()

model = CNNModel(2, 2, 300, sensors, len(adls), nn.ReLU(), nn.Softmax(dim=1))

In [2]:
result = split_and_validate(data=sdf, ground_truth=adf, model=model, labels=adls, label_encoder = adl_encoder)



Training started....
Epoch 1/10, Loss: 8066.517080307007
Epoch 2/10, Loss: 8110.227904319763
Epoch 3/10, Loss: 8107.912800073624
Epoch 4/10, Loss: 8116.657859802246
Epoch 5/10, Loss: 8114.1412189006805
Epoch 6/10, Loss: 8114.074526309967
Epoch 7/10, Loss: 8131.564718008041
Epoch 8/10, Loss: 8265.780719518661
Epoch 9/10, Loss: 8265.780719518661
Epoch 10/10, Loss: 8265.780719518661


Training completed.


In [3]:
import pandas as pd

print('\n')
print('Accuracy =')
print(result[0])
cm = result[1]
cm = pd.DataFrame(cm, columns=adls)
pd.set_option("display.max_columns", 10)
print(cm)




Accuracy =
0.02578616352201258
   Sleeping  Idle  Bathroom_Usage  Meal_Preparation  Relax  Outside
0         0     0               0                 0      0      605
1         0    37               0                 0      0       58
2         0    16               0                 0      0        9
3         0     7               0                 0      0       19
4         0     2               0                 0      0      718
5         0   115               0                 0      0        4
